In [1]:
import numpy as np 
import meshplot as mp
import mitsuba as mi
import drjit as dr

In [2]:
mi.set_variant("cuda_ad_rgb")
# dr.set_flag(dr.JitFlag.VCallRecord, False)
# dr.set_flag(dr.JitFlag.LoopRecord, False)

In [3]:
scene_description = {
    'type': 'scene',
    'heightfield': {
        'type': 'heightfield',
        'filename': 'data/depth.bmp',
        'max_height': 1.0,
        'bsdf': {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': [0.5, 0.5, 0.5]
            }
        }
    },
    'sensor' :{
        'type': 'perspective',
        'to_world': mi.ScalarTransform4f.look_at(
            #origin=[0, 0, 1], target=[0, 0, 0], up=[0, 1, 0]
            origin=[0, -5, 1], target=[0, 0, 0], up=[0, 0, 1]
            #origin=[0, 0, 1], target=[0, 0, 0], up=[0, 1, 0]
        ),
#         'near_clip': 10,
#         'far_clip':2800,
        'film': {
            'type': 'hdrfilm',
            'width': 500,
            'height': 500,
            'sample_border': True
        },
        'sampler': {
            'type': 'independent',
            'sample_count': 2048,
        }
    },
    'integrator': {
        'type': 'direct_reparam',
        'reparam_rays': 128,
        'reparam_antithetic': True,
        'reparam_kappa': 10**6,
    },
    
        
       'sphere_2': {
        'type': 'sphere',
        'center': [0, -7, 1],
        'radius': 1,
       'emitter': {
            'type': 'area',
            'radiance': {
                'type': 'rgb',
                'value': 1.0,
            }
        }
    }    
  
#     'emitter': {
#         'type': 'constant',
#     }
}

In [4]:
scene = mi.load_dict(scene_description)
params = mi.traverse(scene)

In [5]:
heightfield = {
    'res_x': params['heightfield.res_x'],
    'res_y': params['heightfield.res_y'],
    'heightfield': params['heightfield.heightfield'],
    'to_world': params['heightfield.to_world'],
    'max_height': params['heightfield.max_height'],
    'vertex_normals': params['heightfield.per_vertex_normals']
}

In [6]:
def query_normal(idx, heightfield):
    return dr.gather(mi.Normal3f, heightfield['vertex_normals'], idx)

In [7]:
"""
 Given 3 triangle vertices p0, p1 and p2 in world-space, and point p_x, this function determines the barycentric
 UV coordinates of point p_x
"""
def compute_barycentric(p0, p1, p2, p_x):
    dp0 = p1 - p0
    dp1 = p2 - p0
    rel = p_x - p0
    
    bb1 = dr.dot(dp0, rel)
    bb2 = dr.dot(dp1, rel)
    a11 = dr.dot(dp0, dp0)
    a12 = dr.dot(dp0, dp1)
    a22 = dr.dot(dp1, dp1)
    inv_det = dr.rcp(a11 * a22 - a12 * a12)
    
#     u = dr.fmsub(a22, bb1, a12 * bb2) * inv_det
    u = (a22 * bb1 - a12 * bb2) * inv_det
#     v = (-dr.fmadd(a12, bb1, a11 * bb2) * inv_det
    v = (-(a12 * bb1 + a11 * bb2)) * inv_det
    w = 1.0 - u - v
    return w,u,v

In [8]:
"""
    Given the UV coordinates of a point and the UV-space vertices of the 
    tile for which we are checking, this function returns the UV-space vertices of the triangle in which the point
    is located.
   0-------3
    | \pos|
    |  \  |    Outcome mapping
    |neg\ |
   1-------2
"""
def determine_containing_triangle(uv, tile_vertices, tile_vertices_world, tile_normals):
    # We can determine this in 2D, only x and y matters 
    # Sign of determinant: positive: right, negative: left 
    side = dr.sign((tile_vertices[2][0] - tile_vertices[0][0]) * (uv[1] - tile_vertices[0][1]) - (tile_vertices[2][1] - tile_vertices[0][1]) * (uv[0] - tile_vertices[0][0]))
    
    # UV-space vertices
    hit_tri_uv_0_0 = dr.select(side < 0, tile_vertices[0][0], tile_vertices[3][0])
    hit_tri_uv_0_1 = dr.select(side < 0, tile_vertices[0][1], tile_vertices[3][1])

    hit_tri_uv_1_0 = dr.select(side < 0, tile_vertices[1][0], tile_vertices[0][0])
    hit_tri_uv_1_1 = dr.select(side < 0, tile_vertices[1][1], tile_vertices[0][1])
    
    hit_tri_uv_2 = tile_vertices[2]
    
    # World-space vertices
    hit_tri_world_0_0 = dr.select(side < 0, tile_vertices_world[0][0], tile_vertices_world[3][0])
    hit_tri_world_0_1 = dr.select(side < 0, tile_vertices_world[0][1], tile_vertices_world[3][1])
    hit_tri_world_0_2 = dr.select(side < 0, tile_vertices_world[0][2], tile_vertices_world[3][2])

    hit_tri_world_1_0 = dr.select(side < 0, tile_vertices_world[1][0], tile_vertices_world[0][0])
    hit_tri_world_1_1 = dr.select(side < 0, tile_vertices_world[1][1], tile_vertices_world[0][1])
    hit_tri_world_1_2 = dr.select(side < 0, tile_vertices_world[1][2], tile_vertices_world[0][2])
    
    hit_tri_world_2 = tile_vertices_world[2]
    
    # Per-vertex normals
    hit_tri_normals_0_0 = dr.select(side < 0, tile_normals[0][0], tile_normals[3][0])
    hit_tri_normals_0_1 = dr.select(side < 0, tile_normals[0][1], tile_normals[3][1])
    hit_tri_normals_0_2 = dr.select(side < 0, tile_normals[0][2], tile_normals[3][2])

    hit_tri_normals_1_0 = dr.select(side < 0, tile_normals[1][0], tile_normals[0][0])
    hit_tri_normals_1_1 = dr.select(side < 0, tile_normals[1][1], tile_normals[0][1])
    hit_tri_normals_1_2 = dr.select(side < 0, tile_normals[1][2], tile_normals[0][2])
    
    hit_tri_normals_2 = tile_normals[2]
    
    
    return (mi.Vector2f(hit_tri_uv_0_0, hit_tri_uv_0_1), mi.Vector2f(hit_tri_uv_1_0, hit_tri_uv_1_1), mi.Vector2f(hit_tri_uv_2)), \
    (mi.Vector3f(hit_tri_world_0_0, hit_tri_world_0_1, hit_tri_world_0_2), mi.Vector3f(hit_tri_world_1_0, hit_tri_world_1_1, hit_tri_world_1_2), mi.Vector3f(hit_tri_world_2)), \
    (mi.Vector3f(hit_tri_normals_0_0, hit_tri_normals_0_1, hit_tri_normals_0_2), mi.Vector3f(hit_tri_normals_1_0, hit_tri_normals_1_1, hit_tri_normals_1_2), mi.Vector3f(hit_tri_normals_2))


In [9]:
"""
 Given a uv point coordinate (e.g. global heightfield UV of an intersection point), this function returns the
 vertices of the heightfield tile in which the point is located in UV-space, the tile vertices in world
 space, as well the tile normals. 
"""
def uv_to_tile(uv, res_x, res_y, heightfield):
    cell_size = (2.0 / (res_x - 1), 2.0 / (res_y - 1))
    cell_size_uv_space = (1.0 / (res_x - 1), 1.0 / (res_y - 1))
    amount_tiles_x = res_x - 1
    amount_tiles_y = res_y - 1
    
    u_width_per_tile = 1.0 / amount_tiles_x
    v_width_per_tile = 1.0 / amount_tiles_y
    
    tile_x = dr.floor(uv[0] / u_width_per_tile)
    tile_y = dr.floor(uv[1] / v_width_per_tile)
    
    lb_idx = (amount_tiles_y - tile_y) * res_x + tile_x
    rb_idx = (amount_tiles_y - tile_y) * res_x + tile_x + 1
    lt_idx = (amount_tiles_y - (tile_y + 1)) * res_x + tile_x
    rt_idx = (amount_tiles_y - (tile_y + 1)) * res_x + tile_x + 1
    
    local_min_bounds = mi.Point2f(-1.0 + tile_x * cell_size[0], -1.0 + tile_y * cell_size[1])
    local_max_bounds = mi.Point2f(-1.0 + (tile_x + 1) * cell_size[0], -1.0 + (tile_y + 1) * cell_size[1])
    
    local_min_bounds_uv_space = mi.Point2f(tile_x * cell_size_uv_space[0], tile_y * cell_size_uv_space[1])
    local_max_bounds_uv_space = mi.Point2f((tile_x + 1) * cell_size_uv_space[0], (tile_y + 1) * cell_size_uv_space[1])
    
    v0 = (local_min_bounds_uv_space[0], local_max_bounds_uv_space[1])
    v1 = (local_min_bounds_uv_space[0], local_min_bounds_uv_space[1])
    v2 = (local_max_bounds_uv_space[0], local_min_bounds_uv_space[1])
    v3 = (local_max_bounds_uv_space[0], local_max_bounds_uv_space[1])

    v0_world = heightfield['to_world'].transform_affine(mi.Point3f(local_min_bounds[0], local_max_bounds[1], dr.gather(mi.Float, heightfield['heightfield'].array, lt_idx) * heightfield['max_height']))
    v1_world = heightfield['to_world'].transform_affine(mi.Point3f(local_min_bounds[0], local_min_bounds[1], dr.gather(mi.Float, heightfield['heightfield'].array, lb_idx) * heightfield['max_height']))
    v2_world = heightfield['to_world'].transform_affine(mi.Point3f(local_max_bounds[0], local_min_bounds[1], dr.gather(mi.Float, heightfield['heightfield'].array, rb_idx) * heightfield['max_height']))
    v3_world = heightfield['to_world'].transform_affine(mi.Point3f(local_max_bounds[0], local_max_bounds[1], dr.gather(mi.Float, heightfield['heightfield'].array, rt_idx) * heightfield['max_height']))
        
    n0 = query_normal(lt_idx, heightfield)
    n1 = query_normal(lb_idx, heightfield)
    n2 = query_normal(rb_idx, heightfield)
    n3 = query_normal(rt_idx, heightfield)

    # print(f"uv: {uv} tile_vs: {(v0, v1, v2, v3)}")

    #    - tile vertices (UV) -     ---- tile vertices (world) -----     -- tile normals --
    return (v0, v1, v2, v3),    (v0_world, v1_world, v2_world, v3_world), (n0, n1, n2, n3)

In [10]:
def project_uv_on_heightfield(uv, heightfield):
    tile_vertices, tile_vertices_world, tile_normals = uv_to_tile(uv, heightfield['res_x'], heightfield['res_y'], heightfield)
    tri_vertices, tri_vertices_world, tri_normals = determine_containing_triangle(uv, tile_vertices, tile_vertices_world, tile_normals)
    b0, b1, b2 = compute_barycentric(tri_vertices[0], tri_vertices[1], tri_vertices[2], uv)
    
    p_x = b0 * tri_vertices_world[0] + b1 * tri_vertices_world[1] + b2 * tri_vertices_world[2]
    n_x = b0 * tri_normals[0] + b1 * tri_normals[1] + b2 * tri_normals[2]
    return p_x, n_x

In [11]:
def compute_next_delta_clamp_analytic(T0, d, t_prev, epsilon = 0.0):
    delta_x_min = T0[0] / d[0] + t_prev
    delta_x_max = -(((1.0 - T0[0]) / d[0]) - t_prev)
    delta_y_min = T0[1] / d[1] + t_prev
    delta_y_max = -(((1.0 - T0[1]) / d[1]) - t_prev)

    delta_x_min_sign = dr.sign(delta_x_min)
    delta_x_max_sign = dr.sign(delta_x_max)
    delta_y_min_sign = dr.sign(delta_y_min)
    delta_y_max_sign = dr.sign(delta_y_max)

    # The clamping bounds are defined by the minimum of the absolute values of the deltas computed here,
    # because each such delta would bring us out of bounds
    return (dr.select(dr.abs(delta_x_min) < dr.abs(delta_y_min), (abs(delta_x_min) - epsilon) * delta_x_min_sign, (abs(delta_y_min) - epsilon) * delta_y_min_sign),  \
            dr.select(dr.abs(delta_x_max) <= dr.abs(delta_y_max), (abs(delta_x_max) - epsilon) * delta_x_max_sign, (abs(delta_y_max) - epsilon) * delta_y_max_sign))

In [12]:
def project_sample_newtons_method(T0, d, view_O, num_iter, active, min_accuracy = 1e-4, delta_clamp_epsilon = 0.0):
    t = mi.Float(0)
    cnt = mi.UInt32(0)
    active_newton = mi.Mask(active)
    succ = active & False
    loop1 = mi.Loop("Newtons_method", lambda: (cnt, t, active_newton, succ))
    while loop1(active_newton):
        value = compute_F(T0, d, t, view_O, heightfield)
        gradient = F_grad(compute_F, T0, d, t, view_O, heightfield)
        active_newton &= dr.abs(gradient) > 1e-3
        delta = value / gradient
        delta_bounds = compute_next_delta_clamp_analytic(T0, d, t, delta_clamp_epsilon) 
        delta = dr.clamp(delta, delta_bounds[0], delta_bounds[1])
        
        # # Jump back to avoid getting stuck on a corner
        # delta = dr.select(abs(delta) < 1e-3, -(dr.sign(delta)) * 0.8, delta)

        t -= delta
        # print(f"Iteration: {cnt} Value: {value} t: {t} UV-point: {T0 + t * d} gradient: {gradient} delta: {delta} active_newton: {active_newton}")
        cnt += 1
        succ |= active_newton & (dr.abs(value) < min_accuracy)
        active_newton &= (~succ) & (cnt < num_iter)

    return dr.select(succ, t, 0.), succ

In [13]:
"""
    Computes and returns the gradient of the function `func` (in our case `F`)
"""
def F_grad(func, T0, d, t, view_O, heightfield):
    dr.enable_grad(t)
    value = func(T0, d, t, view_O, heightfield)
    dr.forward_from(t)
    grad = dr.grad(value)
    dr.disable_grad(t)
    return grad

In [14]:
"""
    Computes the dot product between surface shading normal and viewing direction,
    which is a metric to determine whether the surface point is on a silhouette.
"""
def compute_F(T0, d, t, view_O, heightfield):
    UV0 = T0 + t * d
    UV0 = dr.clamp(UV0, mi.Point2f(0.0, 0.0), mi.Point2f(1.0, 1.0))
    P, N = project_uv_on_heightfield(UV0, heightfield) 
    V = dr.normalize(view_O - P)
    F = dr.dot(N, V)
    return F

In [15]:
def get_colormap():
    import matplotlib
    cmap_base = matplotlib.colormaps['Spectral']
    N = 10;
    t1 = 0.42;
    cmap_new = cmap_base(np.arange(0, t1+(t1-0)/N, (t1-0)/N))
    cmap_new = np.concatenate((cmap_new, [[0.85, 0.85, 0.85, 1]]), axis=0)  # Gray
    t2 = 0.79;  # t2 = 0.85;
    cmap_new = np.concatenate((cmap_new, cmap_base(np.arange(t2, 1+(1-t2)/N, (1-t2)/N)) ), axis=0)  # Gray
    cmap_colors = np.flipud(cmap_new)
    return matplotlib.colors.LinearSegmentedColormap.from_list("custom_spectral", cmap_colors)

In [16]:
def visualize_newtons_method_roots(roots, vp=None):
    # Surface
    N_res = 300
    x, y = np.meshgrid(np.linspace(0, 1, N_res), np.linspace(0, 1, N_res))
    x = x.flatten()
    y = y.flatten()
    pts, normals = project_uv_on_heightfield(mi.Point2f(x,y), heightfield)
    
    # Strips: densely sampled points along x and y dimension in the unit square to form a grid
    # We have `N_strip` strips along the x-axis, each strip has y-coord [1/(N_strip+1), 2/(N_strip+1), ..., N_strip/(N_strip+1)]
    # We have `N_strip` strips along the y-axis, each strip has x-coord [1/(N_strip+1), 2/(N_strip+1), ..., N_strip/(N_strip+1)]
    # Each strip has `N_res * dense` points
    N_strip = 30
    dense = 5
    t = np.linspace(0, 1, N_res * dense)
    M = N_strip * N_res * dense
    x = np.zeros((2 * M, ))
    y = np.zeros((2 * M, ))
    x[:M] = np.tile(t, N_strip)
    y[M:] = np.tile(t, N_strip)
    y[:M] = np.repeat(np.linspace(1 / (N_strip + 1), N_strip / (N_strip + 1), N_strip), N_res * dense)
    x[M:] = np.repeat(np.linspace(1 / (N_strip + 1), N_strip / (N_strip + 1), N_strip), N_res * dense)
    pts_strip, normals_strip = project_uv_on_heightfield(mi.Point2f(x,y), heightfield)

    v_box = np.array([[-1, -1, 0], [1, -1, 0], [1, 1, 0], [-1, 1, 0],
                    [-1, -1, 1], [1, -1, 1], [1, 1, 1], [-1, 1, 1.]])
    f_box = np.array([[0, 1], [1, 2], [2, 3], [3, 0], [4, 5], [5, 6], [6, 7], 
                    [7, 4], [0, 4], [1, 5], [2, 6], [7, 3]], dtype=int)

    plt = mp.plot(v_box, f_box, shading={"background": "#606060"})
    # plt.add_points(pts.numpy(), shading={"point_size": 0.02, "point_color": '#808080'})
    # plt.add_points(pts_strip.numpy(), shading={"point_size": 0.018, "point_color": '#303060'})

    # Plot roots
    plt.add_points(roots.numpy(), shading={"point_size": 0.2, "point_color": 'black'})

    if vp is not None:
        # Compute normal
        viewdir = dr.normalize(vp - pts)

        if False:  # plot normal
            normals_np = normals.numpy()
            plt.add_lines(pts.numpy(), pts.numpy() + normals_np * 0.1, shading={"line_color": "black", "line_width": 0.1})

        # Plot F function (boundary test)
        normal_viewdir_dot = dr.dot(normals, viewdir).numpy()
        normal_viewdir_dot = (normal_viewdir_dot + 1) / 2
        pts_xy = pts.numpy().copy()
        cmap = get_colormap()
        plt.add_points(pts_xy, shading={"point_size": 0.04,}, c=cmap(normal_viewdir_dot)[:, :3])


        # Plot viewpoint and viewing direction 
        plt.add_points(vp.numpy(), shading={"point_size": 0.2, "point_color": "red"},)
        center = np.array([0.0, 0.0, 0.0])
        to_center = center - vp
        to_center /= np.linalg.norm(to_center)
        v0 = vp + to_center * 0.0
        v1 = vp + to_center * 0.25
        plt.add_lines(v0.numpy(), v1.numpy(), shading={"line_color": "black", "line_width": 4})

    return plt

In [32]:
sampler = mi.load_dict({"type": "independent"})
sampler.seed(7, 100)
s, c = dr.sincos(sampler.next_1d() * dr.two_pi)

# y_T = dr.linspace(mi.Float, 0.2, 0.8, 100)
# view_T = mi.Point2f(0.5, y_T)
# view_O, _ = project_uv_on_heightfield(view_T, heightfield)
view_O = mi.Point3f(-2, 0, 0.2)
view_O_uv = mi.Point2f(-1, 0.5)

if False:
    d = mi.Vector2f(c, s)
    T0 = view_T # + mi.Vector2f(c, s) * 0.3 # TODO: it would be nice if the heightmap intersection routine also returns global UVs
    T0 = dr.clamp(T0, mi.Point2f(0.0, 0.0), mi.Point2f(1.0, 1.0))
else:
    N = 100
    T0 = mi.Point2f(dr.meshgrid(mi.Float(np.arange(0.01, 0.99, 1./N)), mi.Float(np.arange(0.01, 0.99, 1./N))))
    d = dr.normalize(T0 - view_O_uv)

T0_3d, _ = project_uv_on_heightfield(T0, heightfield)

In [33]:
root, succ = project_sample_newtons_method(T0, d, view_O, 20, mi.Mask(True), 1e-5 ,0.0 )

res = T0 + root * d
res[~succ] *= 0
roots_3d, _ = project_uv_on_heightfield(res, heightfield) 

In [35]:
# V0 = mi.Vector3f(0.5, -1.0, 0.4)
plt = visualize_newtons_method_roots(roots_3d, view_O)
# plt.add_points(T0_3d.numpy(), shading={"point_size": 0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…